# K-Shortest Path using Dijkstra
Sia dato un grafo orientato e pesato *G=(V,A)* con *n* vertici, numerati da 1 a *n*, e con archi *(i,j)* di costo non negativo $c_{ij}$. Supponiamo di voler determinare il cammino *P* di costo minimo dal vertice *s* al vertice *d*.

L'algoritmo di **Dijkstra** si serve di una lista in cui vengono memorizzate le coppie vertice-costo $(i, l_i)$, dove *i* è un vertice raggiungibile tramite un cammino semplice con origine in *s* e di costo $l_i$.; per tenere traccia di tale cammino, viene memorizzato per ogni vertice *i* incluso in lista il vertice che lo precede nel cammino da *s* a *i*. 

La lista viene inizializzata inserendo la coppia $(s,0)$. Ad ogni iterazione viene estratta dalla lista la coppia $(i, l_i)$, con costo $l_i$ minore e e vengono presi in considerazione i vertici successivi a *i*, cioè i vertici *j* tali che $(i, j) \in A$; se il vertice *j* non è già presente, in lista viene inserita la coppia $(i, l_i + c_{ij})$, altrimenti si confronta il costo appena ottenuto con quello che era stato precedentemente calcolato e si mantiene quello minore.

 L'algoritmo termina quando il vertice della coppia estratta dalla lista è il vertice *d*. 



Di seguito viene presentato in dettaglio l'Algoritmo di **Dijkstra**.  

In [9]:
import heapq
import numpy as np
import csv
from graph import Graph, Dijkstra, K_Dijkstra
import matplotlib as plt

![Dijsktra](./Images/dijkstra.png)

My python implementation:
```python
def Dijkstra(graph: Graph, start: int, destination: int):
    """_summary_

    Args:
        graph (Graph): Graph, where all the vertex are stored
        start (int): starting vertex
        destination (int): destination vertex

    Returns:
        list : path from start to destination
    """
    if graph.getVertex(start) == None or graph.getVertex(destination) == None:
        return []
    l_i = np.full(graph.getSize()+1, float('inf'))
    l_i[start] = 0
    Pred = np.full(graph.getSize()+1, -1)
    heap = []
    heapq.heapify(heap)
    heapq.heappush(heap, (0, start))
    curr_vertex = start
    while curr_vertex != destination and len(heap) > 0:
        cost, curr_vertex = heapq.heappop(heap)
        if curr_vertex != destination:
            for vertex_value, cost in graph.getVertex(curr_vertex).getneighbors().items():
                distance = l_i[curr_vertex] + cost
                if l_i[vertex_value.getVertex()] > distance:
                    l_i[vertex_value.getVertex()] = distance
                    heapq.heappush(heap, (l_i[vertex_value.getVertex()], vertex_value.getVertex()))
                    Pred[vertex_value.getVertex()] = curr_vertex
        
    def build_path():
        path = [destination]
        print(destination)
        curr_vertex = destination
        while curr_vertex != start:
            curr_vertex = Pred[curr_vertex]
            print(curr_vertex)
            path.append(curr_vertex)
        
        path.reverse()
        return path
        
    return build_path()
```

La funzione ``Heap.Empty()`` crea una lista vuota. La funzione ``Heap.Pop()`` estrae dalla lista la coppia $(j, l_j)$ con $l_j$ minore. La funzione ``Heap.Push``$(j, l_j)$ inserisce la coppia $(j, l_j)$ in lista; nel caso in lista sia già presente una coppia $(j, l_j'$) si mantiene solo il valore minore tra: $l_j$ e $l_j'$. 

In ``Pred(j)`` si memoriza il vertice che precede $j$ nel cammino di costo minore da $s$ a $j$ disponibile fino a quel momento; si tratta del vertice *i* in cui ci si trovava quando è stata aggiunta in lista la coppia $(j, l_j)$.  

Una volta terminato l'algoritmo l'ultima coppia estratta dalla lista sarà del tipo $(d,c)$ e si avrà ``Pred(d)=``$x_{r-1}$, cioè lo *shortest path* da *s* a *d* ha costo *c* e il vertice precedente a *d* in tale cammino è il vertice $x_{r-1}$; per determinare gli altri vertici del cammino si controlla prima ``Pred(``$x_{r-1}$``)`` e si prosegue poi a ritroso. 

Come già anticipato, l'Algoritmo di Dijkstra può anche essere modificato e ampliato per ottenere i primi *K* cammini meno costosi. In questo caso un elemento della lista è il vettore $(j, l, i, k')$: *j* rappresenta un nodo raggiungibile con un cammino di costo *l* a partire da *s* e il cui vertice precedente in tale cammino è il vertice i; $k'$ indica quale tra i cammini meno costosi è quello utilizzato per congiungere $s$ ad $i$. *Pred* diventa una matrice $n \times K$. L'assegnazione $Pred(i, k_i)=(h, k')$ indica che il $k_iesimo$ miglior cammino trovato da *s* a *i* è ottenuto unendo l'arco $(h, i)$ al $k'-esimo$ miglior cammino da *s* a *h*. L'algoritmo diventa:  

![K-Shortest-Path](./Images/k-shortest.png)

My python implementation
```python

def K_Dijkstra(graph: Graph, start: int, destination: int, k: int):
    """_summary_

    Args:
        graph (Graph): Graph, where all the vertex are stored
        start (int): starting Vertex
        destination (int): destination Vertex
        k (int): how many paths must be generated

    Returns:
        list(list): where all the k-paths are stored from start to destination 
    """
    paths = []
    heap = [(0, [], start)]
    while heap and len(paths) < k:
        cost, path, current_node = heapq.heappop(heap)
        path = path + [current_node]
        if current_node and current_node == destination:
            paths.append(path)
        elif graph.getVertex(current_node):
            for neighbor, weight in graph.getVertex(current_node).getneighbors().items():
                if neighbor.getVertex() not in path:
                    heapq.heappush(heap, (cost + weight, path, neighbor.getVertex()))

    return paths
```

In [10]:
graph = Graph()
with open('graph.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        v1 = int(row[0])
        v2 = int(row[1])
        cost = int(row[2])
        graph.addArch(v1, v2, cost)
        print(f"Node : Start {v1}, Neighbor {v2}, Cost {cost}")

print(graph.getSize())

print(f"K-Dijkstra: {K_Dijkstra(graph, 1, 10, 3)}")

Node : Start 1, Neighbor 2, Cost 5
Node : Start 1, Neighbor 3, Cost 9
Node : Start 1, Neighbor 5, Cost 6
Node : Start 2, Neighbor 3, Cost 3
Node : Start 2, Neighbor 6, Cost 2
Node : Start 3, Neighbor 4, Cost 8
Node : Start 3, Neighbor 7, Cost 3
Node : Start 4, Neighbor 5, Cost 1
Node : Start 4, Neighbor 9, Cost 2
Node : Start 5, Neighbor 6, Cost 7
Node : Start 5, Neighbor 10, Cost 4
Node : Start 6, Neighbor 7, Cost 5
Node : Start 6, Neighbor 11, Cost 8
Node : Start 7, Neighbor 8, Cost 1
Node : Start 7, Neighbor 12, Cost 4
Node : Start 8, Neighbor 9, Cost 6
Node : Start 8, Neighbor 13, Cost 2
Node : Start 9, Neighbor 10, Cost 3
Node : Start 9, Neighbor 14, Cost 7
Node : Start 10, Neighbor 11, Cost 9
Node : Start 10, Neighbor 15, Cost 4
Node : Start 11, Neighbor 12, Cost 6
Node : Start 11, Neighbor 16, Cost 3
Node : Start 12, Neighbor 13, Cost 1
Node : Start 12, Neighbor 17, Cost 5
Node : Start 13, Neighbor 14, Cost 4
Node : Start 13, Neighbor 18, Cost 9
Node : Start 14, Neighbor 15, Cos

In [16]:
print(f"Dijkstra: {Dijkstra(graph, 1, 19)}")


19
14
13
8
7
3
2
1
Dijkstra: [1, 2, 3, 7, 8, 13, 14, 19]
